<a href="https://colab.research.google.com/github/owilli38/DSBA6390-Fraud-Detection-Emerging-Fraud-Signals-Dashbaord-/blob/main/Data_Processing_and_Ingestion_Pipelineipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!pip install feedparser pandas

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 1.7 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=aeb28848ca8b6d9600774a21ac9d107bac843e43e0d05c0a1f2fa1d49538ec4f
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [3]:
import feedparser

In [4]:
!pip install supabase feedparser beautifulsoup4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.0/730.0 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 7.0.1
    Uninstalling cachetools-7.0.1:
      Successfully uninstalled cachetools-7.0.1


In [5]:
from supabase import create_client
from google.colab import userdata
from datetime import datetime, timezone

SUPABASE_URL = userdata.get("SUPABASE_URL")
SUPABASE_KEY = userdata.get("SUPABASE_KEY")

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
print("Connected:", SUPABASE_URL[:30])


Connected: https://ecejqerdycocacljrpox.s


In [6]:
import os
from google.colab import userdata
import google.generativeai as genai

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

# Diagnostic print statement: Check if the key is being retrieved
print(f"Retrieved GEMINI_API_KEY (first 5 chars): {GEMINI_API_KEY[:5] if GEMINI_API_KEY else 'None'}")

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

def embed_text(text):
    model = 'models/embedding-001'
    response = genai.embed_content(
        model=model,
        content=text
    )
    return response['embedding']

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


Retrieved GEMINI_API_KEY (first 5 chars): AIzaS


In [7]:
!pip install requests

In [8]:
import requests

url = "https://www.pymnts.com/today-on-pymnts/"
response = requests.get(url)
html_content = response.text

print("Fetched HTML content successfully. Length:", len(html_content))

Fetched HTML content successfully. Length: 309708


In [9]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_content, 'html.parser')
print("HTML content parsed successfully into a BeautifulSoup object.")

HTML content parsed successfully into a BeautifulSoup object.


## Identify Article Elements


In [10]:
ARTICLE_CONTAINER_SELECTOR = "div.col.mb-3.group-0"
TITLE_ELEMENT_SELECTOR = "a.fs-5.fw-bold"
SUMMARY_SELECTOR = "p"

articles_data = []

for article_container in soup.select(ARTICLE_CONTAINER_SELECTOR):
    title_element = article_container.select_one(TITLE_ELEMENT_SELECTOR)
    summary_element = article_container.select_one(SUMMARY_SELECTOR)

    title = title_element.get_text(strip=True) if title_element else ""
    url = title_element['href'] if title_element and 'href' in title_element.attrs else ""
    summary = summary_element.get_text(strip=True) if summary_element else ""

    if title and url and summary: # Only add if all core components are found
        articles_data.append({
            "title": title,
            "url": url,
            "summary": summary
        })

print(f"Extracted {len(articles_data)} articles.")

if articles_data:
    print("\nFirst extracted article:")
    print(articles_data[0])
else:
    print("No articles extracted with the refined selectors.")

Extracted 18 articles.

First extracted article:
{'title': 'AI: The Best Supporting Actor You’ll Never See', 'url': 'https://www.pymnts.com/the-weekender/2026/ai-the-best-supporting-actor-youll-never-see/', 'summary': 'Every Oscars season, Hollywood celebrates “human artistry” while quietly adopting whatever tech makes the...'}


In [11]:
df_articles = pd.DataFrame(articles_data)

print(f"DataFrame created with {len(df_articles)} rows and {len(df_articles.columns)} columns.")
print("First 5 rows of the DataFrame:")
print(df_articles.head())

DataFrame created with 18 rows and 3 columns.
First 5 rows of the DataFrame:
                                               title  \
0     AI: The Best Supporting Actor You’ll Never See   
1  AI Financial Reporting Platform Inscope Raises...   
2  Multi-Agent Systems Move Business AI From Chat...   
3  Consumer Spending Persists Despite Slower Inco...   
4  Telcos Double Down on AI as 90% Report Positiv...   

                                                 url  \
0  https://www.pymnts.com/the-weekender/2026/ai-t...   
1  https://www.pymnts.com/artificial-intelligence...   
2  https://www.pymnts.com/news/artificial-intelli...   
3  https://www.pymnts.com/consumer-insights/2026/...   
4  https://www.pymnts.com/artificial-intelligence...   

                                             summary  
0  Every Oscars season, Hollywood celebrates “hum...  
1  Inscope raised $14.5 million in a Series A fun...  
2  For the last two years, enterprise artificial ...  
3  Consumers continue to shou

## Extract Full Article Text & Publication Date



In [12]:
import requests
from bs4 import BeautifulSoup

def get_article_details(article_url):
    full_text_candidate = ""
    publication_date = None
    try:
        response = requests.get(article_url)
        response.raise_for_status() # Raise an exception for HTTP errors
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract publication date
        published_time_meta = soup.find('meta', property="article:published_time")
        if published_time_meta and 'content' in published_time_meta.attrs:
            publication_date = published_time_meta['content']

        # A comprehensive list of common selectors for main article content
        selectors = [
            'div.td-post-content',
            'div.entry-content',
            'div.single-post-content',
            'article',
            'div[itemprop="articleBody"]',
            'div.post-content',
            'div.article-body',
            'main',
            'div#content',
            'div.story-content',
            'div.post-entry'
        ]

        for selector in selectors:
            content_element = soup.select_one(selector)
            if content_element:
                # Remove script and style elements to clean the text
                for script_or_style in content_element(['script', 'style']):
                    script_or_style.extract()

                current_text = content_element.get_text(separator=' ', strip=True)
                if current_text and len(current_text.split()) > 30:
                    full_text_candidate = current_text
                    break

        # Fallback for full text extraction
        if not full_text_candidate:
            all_paragraphs = soup.find_all('p')
            if all_paragraphs:
                paragraph_texts = [p.get_text(strip=True) for p in all_paragraphs if p.get_text(strip=True)]
                if paragraph_texts:
                    full_text_candidate = ' '.join(paragraph_texts)
                    if len(full_text_candidate.split()) > 30:
                        full_text_candidate = f"WARNING: Generic paragraph extraction used. Content: {full_text_candidate}"
                    else:
                        full_text_candidate = "No substantial article content found even with generic paragraph extraction."
                else:
                    full_text_candidate = "No paragraph elements found on the page."
            else:
                full_text_candidate = "No paragraph elements found on the page."

        if not full_text_candidate:
            full_text_candidate = "No main article content found with any specified selectors or generic fallback."

    except requests.exceptions.RequestException as e:
        return f"Error fetching URL {article_url}: {e}", None
    except Exception as e:
        return f"An unexpected error occurred: {e}", None

    return full_text_candidate, publication_date

# Apply the function to the 'url' column of df_articles
# The .apply(pd.Series) is crucial to expand the tuple returned by get_article_details into separate columns
df_articles[['full_text', 'publication_date']] = df_articles['url'].apply(get_article_details).apply(pd.Series)

print(f"Updated df_articles with 'full_text' and 'publication_date' columns.")
print("First entry's full text snippet:")
# Check if the full_text is not an error message or warning before printing a clean snippet
if not df_articles['full_text'].iloc[0].startswith("Error") and \
   not df_articles['full_text'].iloc[0].startswith("No main article content") and \
   not df_articles['full_text'].iloc[0].startswith("WARNING:"):
    print(df_articles['full_text'].iloc[0][:500] + "...")
else:
    print(df_articles['full_text'].iloc[0]) # Print the full message if it's an error or warning

print("First entry's publication date:")
print(df_articles['publication_date'].iloc[0])
print(f"Full text and publication date extraction complete for {len(df_articles)} articles.")

Updated df_articles with 'full_text' and 'publication_date' columns.
First entry's full text snippet:
AI: The Best Supporting Actor You’ll Never See By PYMNTS | February 21, 2026 | Every Oscars season, Hollywood celebrates “human artistry” while quietly adopting whatever tech makes the deadline less terrifying. This year, artificial intelligence (AI) is the newest crew member: it doesn’t need a trailer, and it definitely won’t complain about craft-services hummus. The only catch? Nobody wants it walking the red carpet. Get the Full Story Complete the form to unlock this article and enjoy unlimit...
First entry's publication date:
2026-02-21T09:00:59+00:00
Full text and publication date extraction complete for 18 articles.


In [13]:
from datetime import datetime, timezone

df_articles['fetch_timestamp'] = datetime.now(timezone.utc).isoformat()

print("Added 'fetch_timestamp' column to df_articles.")
print("First entry's fetch timestamp:")
print(df_articles['fetch_timestamp'].iloc[0])

Added 'fetch_timestamp' column to df_articles.
First entry's fetch timestamp:
2026-02-22T17:53:03.733482+00:00


In [14]:
import re

def clean_article_text(text):
    if not isinstance(text, str):
        return text

    # Remove common boilerplate phrases found in the extracted text
    patterns_to_remove = [
        r'Get the Full Story Complete the form to unlock this article and enjoy unlimited free access to all PYMNTS content — ',
        r'By PYMNTS \| [A-Za-z]+ \d{1,2}, \d{4} \| ',
        r'By PYMNTS \| [A-Za-z]+ \d{1,2}, \d{4} \|',
        r'Get the Full Story Complete the form to unlock this article and enjoy unlimited free access to all PYMNTS content -- ',
        r'Get the Full Story Complete the form to unlock this article and enjoy unlimited free access to all PYMNTS content ',
        r'Get the Full StoryComplete the form to unlock this article and enjoy unlimited free access to all PYMNTS content',
        r'Sign up for our daily newsletter.',
        r'Sign Up For Our Daily Newsletter',
        r'SUBSCRIBE TO THE PYMNTS NEWSLETTER',
        r'Subscribe to the PYMNTS Newsletter',
        r'SIGN UP FOR THE DAILY NEWSLETTER',
        r'Sign up for the Daily Newsletter',
        r'SUBSCRIBE TO THE DAILY NEWSLETTER',
        r'Subscribe to the Daily Newsletter',
        r'PYMNTS is a registered trademark of What’s Next Media & Analytics, LLC'
    ]

    cleaned_text = text
    for pattern in patterns_to_remove:
        cleaned_text = re.sub(pattern, '', cleaned_text, flags=re.IGNORECASE).strip()

    # Remove multiple spaces and replace with a single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    return cleaned_text

# Apply the cleaning function to the 'full_text' column
df_articles['full_text_cleaned'] = df_articles['full_text'].apply(clean_article_text)

print("Cleaned 'full_text' column and created 'full_text_cleaned'.")
print("First entry's cleaned full text snippet:")
print(df_articles['full_text_cleaned'].iloc[0][:500] + "...")
print(f"Number of articles with cleaned text: {len(df_articles)}")

Cleaned 'full_text' column and created 'full_text_cleaned'.
First entry's cleaned full text snippet:
AI: The Best Supporting Actor You’ll Never See Every Oscars season, Hollywood celebrates “human artistry” while quietly adopting whatever tech makes the deadline less terrifying. This year, artificial intelligence (AI) is the newest crew member: it doesn’t need a trailer, and it definitely won’t complain about craft-services hummus. The only catch? Nobody wants it walking the red carpet. no additional logins required. yes Subscribe to our daily newsletter, PYMNTS Today. By completing this form, ...
Number of articles with cleaned text: 18


In [15]:
print(df_articles)

                                                title  \
0      AI: The Best Supporting Actor You’ll Never See   
1   AI Financial Reporting Platform Inscope Raises...   
2   Multi-Agent Systems Move Business AI From Chat...   
3   Consumer Spending Persists Despite Slower Inco...   
4   Telcos Double Down on AI as 90% Report Positiv...   
5   Square Ecosystem Nears 1,000 Partners to Fuel ...   
6   OpenAI Deploys 200 Employees to Build AI Smart...   
7   Google Introduces WebMCP to Give Browser Acces...   
8   Stablecoins This Week: Financial Giants Bet on...   
9   Nvidia Nears $30 Billion Stake in Record-Break...   
10  Supreme Court Tariff Ruling Resets Trade and I...   
11  AWS Denies Report That Service Outage Was Caus...   
12  Earnings Season Made It Clear: Digitize Supply...   
13  Worldline Scales One Commerce Across Europe to...   
14  SoFi and Square Show Why Bank Charters Matter Now   
15  Card Surcharges Turn Checkout Into a Loyalty T...   
16  Discover Network Shows How 

## Create DataFrame and Store Data


In [16]:
articles_to_upsert = df_articles[['title', 'url', 'summary', 'full_text_cleaned', 'publication_date', 'fetch_timestamp']].copy()

print("Selected relevant columns for upsert.")
print(articles_to_upsert.head())

Selected relevant columns for upsert.
                                               title  \
0     AI: The Best Supporting Actor You’ll Never See   
1  AI Financial Reporting Platform Inscope Raises...   
2  Multi-Agent Systems Move Business AI From Chat...   
3  Consumer Spending Persists Despite Slower Inco...   
4  Telcos Double Down on AI as 90% Report Positiv...   

                                                 url  \
0  https://www.pymnts.com/the-weekender/2026/ai-t...   
1  https://www.pymnts.com/artificial-intelligence...   
2  https://www.pymnts.com/news/artificial-intelli...   
3  https://www.pymnts.com/consumer-insights/2026/...   
4  https://www.pymnts.com/artificial-intelligence...   

                                             summary  \
0  Every Oscars season, Hollywood celebrates “hum...   
1  Inscope raised $14.5 million in a Series A fun...   
2  For the last two years, enterprise artificial ...   
3  Consumers continue to shoulder the expansion o...   
4  Telec

In [17]:
articles_to_upsert.to_csv('articles_with_full_text.csv', index=False)
print("DataFrame saved to 'articles_with_full_text.csv'.")

DataFrame saved to 'articles_with_full_text.csv'.


In [18]:
records_to_insert = articles_to_upsert.rename(columns={
    'summary': 'summary_content',
    'full_text_cleaned': 'full_text_content'
}).to_dict(orient='records')

print(f"Prepared {len(records_to_insert)} records for Supabase upsert.")
print("First record prepared for upsert:")
print(records_to_insert[0])

Prepared 18 records for Supabase upsert.
First record prepared for upsert:
{'title': 'AI: The Best Supporting Actor You’ll Never See', 'url': 'https://www.pymnts.com/the-weekender/2026/ai-the-best-supporting-actor-youll-never-see/', 'summary_content': 'Every Oscars season, Hollywood celebrates “human artistry” while quietly adopting whatever tech makes the...', 'full_text_content': 'AI: The Best Supporting Actor You’ll Never See Every Oscars season, Hollywood celebrates “human artistry” while quietly adopting whatever tech makes the deadline less terrifying. This year, artificial intelligence (AI) is the newest crew member: it doesn’t need a trailer, and it definitely won’t complain about craft-services hummus. The only catch? Nobody wants it walking the red carpet. no additional logins required. yes Subscribe to our daily newsletter, PYMNTS Today. By completing this form, you agree to receive marketing communications from PYMNTS and to the sharing of your information with our sponsor,

## Prepare Data for Supabase (articles_v1 schema)


In [19]:
import hashlib

df_articles_v1 = df_articles[[
    'title',
    'url',
    'publication_date',
    'full_text_cleaned',
    'fetch_timestamp'
]].copy()

df_articles_v1 = df_articles_v1.rename(columns={
    'publication_date': 'publish_timestamp',
    'full_text_cleaned': 'raw_text'
})

df_articles_v1['source'] = 'PYMNTS'
df_articles_v1['source_type'] = 'news'
df_articles_v1['raw_html'] = df_articles_v1['url']

df_articles_v1['doc_id'] = df_articles_v1.apply(
    lambda row: hashlib.sha256((row['source'] + row['url']).encode('utf-8')).hexdigest(),
    axis=1
)

print("Created df_articles_v1 with the new schema.")
print(df_articles_v1.head())

Created df_articles_v1 with the new schema.
                                               title  \
0     AI: The Best Supporting Actor You’ll Never See   
1  AI Financial Reporting Platform Inscope Raises...   
2  Multi-Agent Systems Move Business AI From Chat...   
3  Consumer Spending Persists Despite Slower Inco...   
4  Telcos Double Down on AI as 90% Report Positiv...   

                                                 url  \
0  https://www.pymnts.com/the-weekender/2026/ai-t...   
1  https://www.pymnts.com/artificial-intelligence...   
2  https://www.pymnts.com/news/artificial-intelli...   
3  https://www.pymnts.com/consumer-insights/2026/...   
4  https://www.pymnts.com/artificial-intelligence...   

           publish_timestamp  \
0  2026-02-21T09:00:59+00:00   
1  2026-02-21T00:21:50+00:00   
2  2026-02-20T23:01:07+00:00   
3  2026-02-20T21:41:00+00:00   
4  2026-02-20T20:47:00+00:00   

                                            raw_text  \
0  AI: The Best Supporting Actor 

## Upsert Data to Supabase articles_v1



In [20]:
records_articles_v1 = df_articles_v1.to_dict(orient='records')

print(f"Converted {len(records_articles_v1)} records for articles_v1 table.")
print("First record prepared for articles_v1 upsert:")
print(records_articles_v1[0])

Converted 18 records for articles_v1 table.
First record prepared for articles_v1 upsert:
{'title': 'AI: The Best Supporting Actor You’ll Never See', 'url': 'https://www.pymnts.com/the-weekender/2026/ai-the-best-supporting-actor-youll-never-see/', 'publish_timestamp': '2026-02-21T09:00:59+00:00', 'raw_text': 'AI: The Best Supporting Actor You’ll Never See Every Oscars season, Hollywood celebrates “human artistry” while quietly adopting whatever tech makes the deadline less terrifying. This year, artificial intelligence (AI) is the newest crew member: it doesn’t need a trailer, and it definitely won’t complain about craft-services hummus. The only catch? Nobody wants it walking the red carpet. no additional logins required. yes Subscribe to our daily newsletter, PYMNTS Today. By completing this form, you agree to receive marketing communications from PYMNTS and to the sharing of your information with our sponsor, if applicable, in accordance with our Privacy Policy and Terms and Conditi

In [21]:
response = supabase.table('articles_v1').upsert(records_articles_v1, on_conflict='url').execute()

print(f"Upserted {len(records_articles_v1)} records to Supabase 'articles_v1' table.")
if hasattr(response, 'data'):
    print(f"Supabase response: {len(response.data)} records processed.")
else:
    print(f"Supabase response (error or unexpected format): {response}")

Upserted 18 records to Supabase 'articles_v1' table.
Supabase response: 18 records processed.


## Create Embeddings

In [22]:
response = supabase.table('articles_v1').select('doc_id', 'raw_text').execute()
articles_for_embedding = response.data

print(f"Retrieved {len(articles_for_embedding)} articles from Supabase.")
if articles_for_embedding:
    print("First retrieved article (doc_id and raw_text snippet):")
    print(f"Doc ID: {articles_for_embedding[0]['doc_id']}")
    print(f"Raw Text: {articles_for_embedding[0]['raw_text'][:100]}...")

Retrieved 76 articles from Supabase.
First retrieved article (doc_id and raw_text snippet):
Doc ID: 88aa0eb50a5cbb3f77bb7c2e9ec182c69ebbbc7bb3ae84e4ce2bf512f0a6b38f
Raw Text: Robots, Compliance and AI Receptionists: $1.45 Billion Backs AI Embedded in Real Operations Across r...


In [23]:
df_articles_for_embedding = pd.DataFrame(articles_for_embedding)

print(f"DataFrame created with {len(df_articles_for_embedding)} rows and {len(df_articles_for_embedding.columns)} columns.")
print("First 5 rows of the DataFrame:")
print(df_articles_for_embedding.head())

DataFrame created with 76 rows and 2 columns.
First 5 rows of the DataFrame:
                                              doc_id  \
0  88aa0eb50a5cbb3f77bb7c2e9ec182c69ebbbc7bb3ae84...   
1  8ea00ced265f40e17280642cfc05985224f914ab22567f...   
2  0f9ac36b6ed2ad675fc2be860db33fe45628916cb79ecc...   
3  e1947661540928d996ac0abcafa9f076814d97418d86c7...   
4  c0fb7f44fd628005c2a92da3c3b8fc9e78e63cbdbcfd03...   

                                            raw_text  
0  Robots, Compliance and AI Receptionists: $1.45...  
1  JPMorgan Chase Analysts Predict AI Costs Could...  
2  Fed Governor Michael Barr Says Firms Adopting ...  
3  CFTC Asserts Authority to Police Prediction Ma...  
4  British Banks Push For UK Alternative to Visa ...  


In [24]:
!pip install -q google-generativeai


In [25]:
import os
from google.colab import userdata
import google.generativeai as genai

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

# Diagnostic print statement: Check if the key is being retrieved
print(f"Retrieved GEMINI_API_KEY (first 5 chars): {GEMINI_API_KEY[:5] if GEMINI_API_KEY else 'None'}")

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

def embed_text(text):
    model = 'models/embedding-001'
    response = genai.embed_content(
        model=model,
        content=text
    )
    return response['embedding']

Retrieved GEMINI_API_KEY (first 5 chars): AIzaS


In [26]:
import google.generativeai as genai

for m in genai.list_models():
    if 'embedContent' in m.supported_generation_methods:
        print(m.name)

models/gemini-embedding-001


In [27]:
from google import genai

client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

def embed_text(text):
    response = client.models.embed_content(
        model="models/gemini-embedding-001",
        contents=text,
    )
    return response.embeddings[0].values


if not df_articles_for_embedding.empty:
    sample_text = df_articles_for_embedding['raw_text'].iloc[0]
    try:
        embedding = embed_text(sample_text)
        print(f"Embedding generated successfully. Length: {len(embedding)}")
        print(f"First 10 values of embedding: {embedding[:10]}")
    except Exception as e:
        print(f"Error generating embedding: {e}")
else:
    print("df_articles_for_embedding is empty, cannot test embedding function.")

Embedding generated successfully. Length: 3072
First 10 values of embedding: [0.0015488467, -0.013421287, 0.020624714, -0.09298411, -0.050761152, 0.007767184, 0.003774711, 0.012723935, 0.0049771513, 0.014544116]


In [28]:
df_articles_for_embedding['embedding'] = df_articles_for_embedding['raw_text'].apply(embed_text)

print("Generated embeddings for all articles.")
print("First entry's embedding snippet (first 10 elements):")
print(df_articles_for_embedding['embedding'].iloc[0][:10])
print(f"Embedding shape: {len(df_articles_for_embedding['embedding'].iloc[0])}")

Generated embeddings for all articles.
First entry's embedding snippet (first 10 elements):
[0.0015488467, -0.013421287, 0.020624714, -0.09298411, -0.050761152, 0.007767184, 0.003774711, 0.012723935, 0.0049771513, 0.014544116]
Embedding shape: 3072


## Prepare Embeddings for Supabase Vector



In [29]:
embeddings_to_upsert = df_articles_for_embedding[['doc_id', 'embedding']].to_dict(orient='records')

print(f"Prepared {len(embeddings_to_upsert)} embedding records for Supabase upsert.")
if embeddings_to_upsert:
    print("First embedding record prepared for upsert:")
    print(embeddings_to_upsert[0])
else:
    print("No embedding records prepared.")

Prepared 76 embedding records for Supabase upsert.
First embedding record prepared for upsert:
{'doc_id': '88aa0eb50a5cbb3f77bb7c2e9ec182c69ebbbc7bb3ae84e4ce2bf512f0a6b38f', 'embedding': [0.0015488467, -0.013421287, 0.020624714, -0.09298411, -0.050761152, 0.007767184, 0.003774711, 0.012723935, 0.0049771513, 0.014544116, -0.022750938, 0.030442374, -0.00051530055, 0.002149079, 0.11433324, 0.04259217, -0.0030273984, 0.008100612, -0.0010765606, -0.010160025, -0.0018761128, 0.004516747, -0.020906119, -0.03733846, -0.020167794, 0.0029474746, 0.02983356, 0.020363811, 0.034719873, -0.013346176, 0.004265491, -0.017869323, 0.0139642535, 0.011539918, 0.008861467, 0.025533209, 0.015948994, -0.020735504, -0.015959715, 0.015005814, -0.010199576, -0.006939271, -0.0030319213, -0.002866005, -0.012358501, -0.017191336, 0.026759397, -0.03159759, -0.0044658137, 0.028734714, -0.013809899, -0.009683056, -0.021662781, -0.14726356, -0.0038024706, 0.00018452326, 0.009729443, -0.0019914412, 0.029958155, 0.00382

In [30]:
response = supabase.table('article_embeddings').upsert(embeddings_to_upsert, on_conflict='doc_id').execute()

print(f"Upserted {len(embeddings_to_upsert)} embedding records to Supabase 'article_embeddings' table.")
if hasattr(response, 'data'):
    print(f"Supabase response: {len(response.data)} records processed.")
else:
    print(f"Supabase response (error or unexpected format): {response}")

Upserted 76 embedding records to Supabase 'article_embeddings' table.
Supabase response: 76 records processed.


## Fetch and Parse ACFE Blog HTML


In [31]:
acfe_url = 'https://www.acfe.com/acfe-insights-blog'
response = requests.get(acfe_url)
acfe_html_content = response.text

print(f"Fetched HTML content successfully from {acfe_url}. Length: {len(acfe_html_content)}")

acfe_soup = BeautifulSoup(acfe_html_content, 'html.parser')
print("ACFE HTML content parsed successfully into a BeautifulSoup object.")

Fetched HTML content successfully from https://www.acfe.com/acfe-insights-blog. Length: 1049663
ACFE HTML content parsed successfully into a BeautifulSoup object.


## Identify Article Elements for ACFE Blog



In [32]:
print(acfe_soup.prettify())

<!DOCTYPE html>
<html class="no-js" lang="en">
 <head>
  <meta content="Association of Certified Fraud Examiners" name="author"/>
  <meta content="" name="copyright"/>
  <meta content="" name="description"/>
  <meta content="" name="google-site-verification"/>
  <meta content="" name="keywords"/>
  <meta content="follow,index" name="robots"/>
  <meta content="width=device-width,initial-scale=1.0" name="viewport"/>
  <title>
   ACFE Insights Blog
  </title>
  <script>
   window.dataLayer = window.dataLayer || [];
  </script>
  <!-- Google Tag Manager -->
  <script>
   (function (w, d, s, l, i) {
        w[l] = w[l] || []; w[l].push({
            'gtm.start':
                new Date().getTime(), event: 'gtm.js'
        }); var f = d.getElementsByTagName(s)[0],
            j = d.createElement(s), dl = l != 'dataLayer' ? '&l=' + l : ''; j.async = true; j.src =
                'https://www.googletagmanager.com/gtm.js?id=' + i + dl; f.parentNode.insertBefore(j, f);
    })(window, document, 

## Extract Titles and URLs from ACFE Blog Listing


In [33]:
ACFE_ARTICLE_CONTAINER_SELECTOR = "article.card.blog-card, article.content-card.post-article, article.post"
ACFE_TITLE_ELEMENT_SELECTOR = "h3 a"

acfe_articles_data = []

for article_container in acfe_soup.select(ACFE_ARTICLE_CONTAINER_SELECTOR):
    title_element = article_container.select_one(ACFE_TITLE_ELEMENT_SELECTOR)

    title = title_element.get_text(strip=True) if title_element else ""
    url = title_element['href'] if title_element and 'href' in title_element.attrs else ""
    summary = "" # Summary will be generated from full text later

    # Ensure the URL is absolute if it's relative
    if url and not url.startswith('http'):
        url = 'https://www.acfe.com' + url

    if title and url: # Only add if title and URL are found
        acfe_articles_data.append({
            "title": title,
            "url": url,
            "summary": summary
        })

print(f"Extracted {len(acfe_articles_data)} articles from ACFE blog.")

if acfe_articles_data:
    print("\nFirst extracted ACFE article:")
    print(acfe_articles_data[0])
else:
    print("No ACFE articles extracted with the specified selectors.")

Extracted 4 articles from ACFE blog.

First extracted ACFE article:
{'title': 'Staying Safe This Tax Season', 'url': 'https://www.acfe.com/acfe-insights-blog/blog-detail?s=staying-safe-tax-season', 'summary': ''}


In [34]:
df_acfe_articles = pd.DataFrame(acfe_articles_data)

print(f"DataFrame created with {len(df_acfe_articles)} rows and {len(df_acfe_articles.columns)} columns.")
print("First 5 rows of the DataFrame:")
print(df_acfe_articles.head())

DataFrame created with 4 rows and 3 columns.
First 5 rows of the DataFrame:
                                               title  \
0                       Staying Safe This Tax Season   
1  The Challenges of Reporting Misconduct in Mexi...   
2              Overtime: The Fraud That Does Not Pay   
3  A New Frontier of Financial Crime: What FinCEN...   

                                                 url summary  
0  https://www.acfe.com/acfe-insights-blog/blog-d...          
1  https://www.acfe.com/acfe-insights-blog/blog-d...          
2  https://www.acfe.com/acfe-insights-blog/blog-d...          
3  https://www.acfe.com/acfe-insights-blog/blog-d...          


In [35]:
import requests
from bs4 import BeautifulSoup

def get_article_details_acfe(article_url):
    full_text_candidate = ""
    publication_date = None
    try:
        response = requests.get(article_url)
        response.raise_for_status() # Raise an exception for HTTP errors
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract publication date - ACFE often uses schema.org or similar for dates
        date_element = soup.find('time', class_='datetime')
        if date_element and 'datetime' in date_element.attrs:
            publication_date = date_element['datetime']
        else:
            # Fallback for other date formats if needed, or if datetime is not always available
            meta_published = soup.find('meta', property='article:published_time')
            if meta_published and 'content' in meta_published.attrs:
                publication_date = meta_published['content']

        # A comprehensive list of common selectors for main article content on ACFE
        selectors = [
            'div.main-content article.post',
            'div.body-content',
            'div.body-content.richtext-content',
            'div.field--name-body',
            'div.node__content',
            'article',
            'div[itemprop="articleBody"]',
            'div.post-content'
        ]

        for selector in selectors:
            content_element = soup.select_one(selector)
            if content_element:
                # Remove script and style elements to clean the text
                for script_or_style in content_element(['script', 'style']):
                    script_or_style.extract()

                current_text = content_element.get_text(separator=' ', strip=True)
                if current_text and len(current_text.split()) > 30:
                    full_text_candidate = current_text
                    break

        # Fallback for full text extraction if specific selectors fail
        if not full_text_candidate:
            all_paragraphs = soup.find_all('p')
            if all_paragraphs:
                paragraph_texts = [p.get_text(strip=True) for p in all_paragraphs if p.get_text(strip=True)]
                if paragraph_texts:
                    full_text_candidate = ' '.join(paragraph_texts)
                    if len(full_text_candidate.split()) <= 30:
                        full_text_candidate = "No substantial article content found even with generic paragraph extraction."
                else:
                    full_text_candidate = "No paragraph elements found on the page."
            else:
                full_text_candidate = "No paragraph elements found on the page."

        if not full_text_candidate:
            full_text_candidate = "No main article content found with any specified selectors or generic fallback."

    except requests.exceptions.RequestException as e:
        return f"Error fetching URL {article_url}: {e}", None
    except Exception as e:
        return f"An unexpected error occurred: {e}", None

    return full_text_candidate, publication_date

# Apply the function to the 'url' column of df_acfe_articles
df_acfe_articles[['full_text', 'publication_date']] = df_acfe_articles['url'].apply(get_article_details_acfe).apply(pd.Series)

print(f"Updated df_acfe_articles with 'full_text' and 'publication_date' columns.")
print("First entry's full text snippet:")
# Check if the full_text is not an error message or warning before printing a clean snippet
if not df_acfe_articles['full_text'].empty and not df_acfe_articles['full_text'].iloc[0].startswith("Error") and \
   not df_acfe_articles['full_text'].iloc[0].startswith("No main article content") and \
   not df_acfe_articles['full_text'].iloc[0].startswith("WARNING:"):
    print(df_acfe_articles['full_text'].iloc[0][:500] + "...")
else:
    print(df_acfe_articles['full_text'].iloc[0]) # Print the full message if it's an error or warning

print("First entry's publication date:")
print(df_acfe_articles['publication_date'].iloc[0])
print(f"Full text and publication date extraction complete for {len(df_acfe_articles)} articles.")

Updated df_acfe_articles with 'full_text' and 'publication_date' columns.
First entry's full text snippet:
Become a member of the ACFE. Don't lose your member benefits: Renew today. Complete guide to earning your CFE credential. Learn more about the steps to become certified. Find news and analysis on the fight against fraud. Read case studies, thought leadership and more in the ACFE's digital publication. Connect with the world's largest network of anti-fraud professionals. Find and reach out to other members of the ACFE. Discover the ACFE's nonprofit that supports students pursuing fraud examinatio...
First entry's publication date:
None
Full text and publication date extraction complete for 4 articles.


In [36]:
from datetime import datetime, timezone

df_acfe_articles['fetch_timestamp'] = datetime.now(timezone.utc).isoformat()

print("Added 'fetch_timestamp' column to df_acfe_articles.")
print("First entry's fetch timestamp:")
print(df_acfe_articles['fetch_timestamp'].iloc[0])

Added 'fetch_timestamp' column to df_acfe_articles.
First entry's fetch timestamp:
2026-02-22T17:53:45.043145+00:00


In [37]:
import re

def clean_article_text(text):
    if not isinstance(text, str):
        return text

    # Remove common boilerplate phrases found in the extracted text
    # Adapt patterns specific to ACFE if necessary, otherwise use generic ones
    patterns_to_remove = [
        r'Become a member of the ACFE. Don\'t lose your member benefits: Renew today. Complete guide to earning your CFE credential. Learn more about the steps to become certified. Find news and analysis on the fight against fraud. Read case studies, thought leadership and more in the ACFE\'s digital publication. Connect with the world\'s largest network of anti-fraud professionals. Find and reach out to other members of the ACFE. Discover the ACFE\'s nonprofit that supports students pursuing fraud examinatio...',
        r'Association of Certified Fraud Examiners \| ACFE Insights Blog',
        r'Become a member of the ACFE. Don\'t lose your member benefits: Renew today.',
        r'Complete guide to earning your CFE credential. Learn more about the steps to become certified.',
        r'Find news and analysis on the fight against fraud. Read case studies, thought leadership and more in the ACFE\'s digital publication.',
        r'Connect with the world\'s largest network of anti-fraud professionals. Find and reach out to other members of the ACFE.',
        r'Discover the ACFE\'s nonprofit that supports students pursuing fraud examination. Learn about the ACFE Foundation.',
        r'Home \/ ACFE Insights Blog',
        r'Comments: \d+',
        r'Tags:',
        r'ACFE Insights blog posts are published twice weekly. If you would like to submit a post for consideration, please review our submission guidelines. Submission Guidelines',
        r'By \s*[^\|]* \s*\|\s* [A-Za-z]+ \d{1,2}, \d{4} \s*\|\s*',
        r'Copyright \d{4} Association of Certified Fraud Examiners, Inc. All rights reserved.',
        r'Become a member of the ACFE.',
        r'Don\'t lose your member benefits: Renew today.',
        r'Complete guide to earning your CFE credential.',
        r'Learn more about the steps to become certified.',
        r'Find news and analysis on the fight against fraud.',
        r'Read case studies, thought leadership and more in the ACFE\'s digital publication.',
        r'Connect with the world\'s largest network of anti-fraud professionals.',
        r'Find and reach out to other members of the ACFE.',
        r'Discover the ACFE\'s nonprofit that supports students pursuing fraud examination. Learn about the ACFE Foundation.',
        r'If you\'re looking for career opportunities in the anti-fraud community, the ACFE Career Center is for you. Learn More',
        r'See all events',
        r'Connect and Grow with the ACFE',
        r'Contact Us',
        r'Advertise with Us',
        r'Privacy Policy',
        r'Terms of Use',
        r'Site Map'
    ]

    cleaned_text = text
    for pattern in patterns_to_remove:
        cleaned_text = re.sub(pattern, '', cleaned_text, flags=re.IGNORECASE).strip()

    # Remove multiple spaces and replace with a single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    return cleaned_text

# Apply the cleaning function to the 'full_text' column
df_acfe_articles['full_text_cleaned'] = df_acfe_articles['full_text'].apply(clean_article_text)

print("Cleaned 'full_text' column and created 'full_text_cleaned' for ACFE articles.")
print("First entry's cleaned full text snippet:")
print(df_acfe_articles['full_text_cleaned'].iloc[0][:500] + "...")
print(f"Number of ACFE articles with cleaned text: {len(df_acfe_articles)}")

Cleaned 'full_text' column and created 'full_text_cleaned' for ACFE articles.
First entry's cleaned full text snippet:
areers and delivers the latest anti-fraud research. Decide which study option is best for you. Practical information to help you fight fraud. Dynamic training for your anti-fraud team. Connect with an ACFE member and elevate your career. Visit the job board to discover current opportunities. In this blog, ACFE Training Director Jason Zirkle, CFE, shares practical tips on spotting potential scams and identity theft threats, explains why filing taxes early reduces your fraud risk and offers other ...
Number of ACFE articles with cleaned text: 4


In [41]:
import hashlib

df_acfe_articles_v1 = df_acfe_articles[[
    'title',
    'url',
    'publication_date',
    'full_text_cleaned',
    'fetch_timestamp'
]].copy()

df_acfe_articles_v1 = df_acfe_articles_v1.rename(columns={
    'publication_date': 'publish_timestamp',
    'full_text_cleaned': 'raw_text'
})

df_acfe_articles_v1['source'] = 'ACFE'
df_acfe_articles_v1['source_type'] = 'blog' # Assuming ACFE is a blog
df_acfe_articles_v1['raw_html'] = df_acfe_articles_v1['url'] # raw_html is just the URL for now

df_acfe_articles_v1['doc_id'] = df_acfe_articles_v1.apply(
    lambda row: hashlib.sha256((row['source'] + row['url']).encode('utf-8')).hexdigest(),
    axis=1
)

records_acfe_articles_v1 = df_acfe_articles_v1.to_dict(orient='records')

print(f"Prepared {len(records_acfe_articles_v1)} records for articles_v1 table (ACFE).")
if records_acfe_articles_v1:
    print("First record prepared for articles_v1 upsert (ACFE):")
    print(records_acfe_articles_v1[0])
else:
    print("No records prepared for ACFE articles_v1 upsert.")

Prepared 4 records for articles_v1 table (ACFE).
First record prepared for articles_v1 upsert (ACFE):
{'title': 'Staying Safe This Tax Season', 'url': 'https://www.acfe.com/acfe-insights-blog/blog-detail?s=staying-safe-tax-season', 'publish_timestamp': None, 'raw_text': "areers and delivers the latest anti-fraud research. Decide which study option is best for you. Practical information to help you fight fraud. Dynamic training for your anti-fraud team. Connect with an ACFE member and elevate your career. Visit the job board to discover current opportunities. In this blog, ACFE Training Director Jason Zirkle, CFE, shares practical tips on spotting potential scams and identity theft threats, explains why filing taxes early reduces your fraud risk and offers other advice to help you avoid becoming a victim. Fraudsters are always searching for timely opportunities to exploit people for financial gain. It’s no surprise that tax season provides a significant opportunity to target unsuspectin

In [42]:
response = supabase.table('articles_v1').upsert(records_acfe_articles_v1, on_conflict='url').execute()

print(f"Upserted {len(records_acfe_articles_v1)} records to Supabase 'articles_v1' table (ACFE).")
if hasattr(response, 'data'):
    print(f"Supabase response: {len(response.data)} records processed.")
else:
    print(f"Supabase response (error or unexpected format): {response}")

Upserted 4 records to Supabase 'articles_v1' table (ACFE).
Supabase response: 4 records processed.


In [43]:
response_acfe = supabase.table('articles_v1').select('doc_id', 'raw_text').eq('source', 'ACFE').execute()
articles_for_embedding_acfe = response_acfe.data

print(f"Retrieved {len(articles_for_embedding_acfe)} ACFE articles from Supabase for embedding.")
if articles_for_embedding_acfe:
    print("First retrieved ACFE article (doc_id and raw_text snippet):")
    print(f"Doc ID: {articles_for_embedding_acfe[0]['doc_id']}")
    print(f"Raw Text: {articles_for_embedding_acfe[0]['raw_text'][:100]}...")
else:
    print("No ACFE articles retrieved for embedding.")

Retrieved 4 ACFE articles from Supabase for embedding.
First retrieved ACFE article (doc_id and raw_text snippet):
Doc ID: ba14fa9af4fcf6a0b4eb708b8da22099f00a2afa3b2ec06df874a77d74162aca
Raw Text: areers and delivers the latest anti-fraud research. Decide which study option is best for you. Pract...


In [44]:
df_acfe_articles_for_embedding = pd.DataFrame(articles_for_embedding_acfe)

print(f"DataFrame created with {len(df_acfe_articles_for_embedding)} rows and {len(df_acfe_articles_for_embedding.columns)} columns.")
print("First 5 rows of the DataFrame:")
print(df_acfe_articles_for_embedding.head())

DataFrame created with 4 rows and 2 columns.
First 5 rows of the DataFrame:
                                              doc_id  \
0  ba14fa9af4fcf6a0b4eb708b8da22099f00a2afa3b2ec0...   
1  edd8b9161cb615d2666935032d22289a7d8eadbad17535...   
2  423dd80cccf10f498ad2b723390a50952d3fa1f7a73da7...   
3  484a2430c432663648b184b53fa357600e3f0768fe1968...   

                                            raw_text  
0  areers and delivers the latest anti-fraud rese...  
1  areers and delivers the latest anti-fraud rese...  
2  areers and delivers the latest anti-fraud rese...  
3  areers and delivers the latest anti-fraud rese...  


In [45]:
df_acfe_articles_for_embedding['embedding'] = df_acfe_articles_for_embedding['raw_text'].apply(embed_text)

print("Generated embeddings for all ACFE articles.")
print("First entry's embedding snippet (first 10 elements):")
print(df_acfe_articles_for_embedding['embedding'].iloc[0][:10])
print(f"Embedding shape: {len(df_acfe_articles_for_embedding['embedding'].iloc[0])}")

Generated embeddings for all ACFE articles.
First entry's embedding snippet (first 10 elements):
[0.019793222, 0.013930543, 0.0327192, -0.036517676, -0.020206194, -0.0043040123, 0.026908558, -0.01129642, -0.011367932, 0.0097380085]
Embedding shape: 3072


In [46]:
embeddings_to_upsert_acfe = df_acfe_articles_for_embedding[['doc_id', 'embedding']].to_dict(orient='records')

print(f"Prepared {len(embeddings_to_upsert_acfe)} embedding records for Supabase upsert (ACFE).")
if embeddings_to_upsert_acfe:
    print("First embedding record prepared for upsert (ACFE):")
    print(embeddings_to_upsert_acfe[0])
else:
    print("No embedding records prepared for ACFE.")

Prepared 4 embedding records for Supabase upsert (ACFE).
First embedding record prepared for upsert (ACFE):
{'doc_id': 'ba14fa9af4fcf6a0b4eb708b8da22099f00a2afa3b2ec06df874a77d74162aca', 'embedding': [0.019793222, 0.013930543, 0.0327192, -0.036517676, -0.020206194, -0.0043040123, 0.026908558, -0.01129642, -0.011367932, 0.0097380085, -0.020861816, -0.011953602, 0.0074226963, -0.006778169, 0.11956292, -0.019031204, 0.017404353, -0.0004216277, 0.0110501, -0.017646635, -0.007759577, 0.0014498067, -0.017554436, 0.012422152, 0.0035708135, -0.021176415, 0.013288239, -0.014996812, -0.00039243317, 0.027074248, -0.010631942, 0.006860355, -0.02205931, -0.0046973997, -0.005369185, -0.012956924, 0.006881728, -0.0419607, 0.006981954, -0.011909976, -0.012227511, -0.0053506726, -0.016145485, 0.0019863893, 0.017933179, -0.013378098, 0.002708979, -0.00017228228, -0.017646533, 0.024878776, 0.011589846, 0.008331338, -0.037113328, -0.18069348, 0.006388911, -0.014695252, 0.0065000947, 0.016613765, 0.0201777

In [47]:
response = supabase.table('article_embeddings').upsert(embeddings_to_upsert_acfe, on_conflict='doc_id').execute()

print(f"Upserted {len(embeddings_to_upsert_acfe)} embedding records to Supabase 'article_embeddings' table (ACFE).")
if hasattr(response, 'data'):
    print(f"Supabase response: {len(response.data)} records processed.")
else:
    print(f"Supabase response (error or unexpected format): {response}")

Upserted 4 embedding records to Supabase 'article_embeddings' table (ACFE).
Supabase response: 4 records processed.
